<a href="https://colab.research.google.com/github/debora28/pi-2020.2/blob/main/07_12_Crime_E2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TAXA DE OCORRÊNCIA E PROBABILIDADE DE SOFRER UM CRIME DE ROUBO OU FURTO EM DETERMINADAS CONDIÇÕES
*Os dados foram extraídos do sistema de Registro Digital de Ocorrências (R.D.O.) disponível no site www.ssp.sp.gov.br referente ao ano de 2014.*

Obs: Não foi possível utilizar o mesmo dataset do trabalho anterior por aquele não conter dados importantes para a elaboração deste modelo. Isso pode ser verificado pela ausência das colunas HORA_OCORRENCIA_BO, SEXO_PESSOA, IDADE_PESSOA, DESCR_PROFISSAO e DESCR_GRAU_INSTRUCAO no link: https://colab.research.google.com/drive/1Hjj_3MDJrdZ9yuqeZZpsCA-xdXjo_KTL?usp=sharing

Equipe: Débora Caldas, Renan Urbano, Matheus Guerra, Horlando Leão, Héber Teófilo, Pablo Danillo, Rodrigo Vilemen.

Siglas e termos

*DECAP = Departamento de Polícia Judiciária da Capital;

Descrição das colunas
* NOME_MUNICIPIO_CIRC - Município de Circunscrição
* DESCR_TIPO_BO - Tipo de Documento
* DATA_OCORRENCIA_BO - Data da Ocorrência
* HORA_OCORRENCIA_BO - Hora da Ocorrência
* DATAHORA_COMUNICACAO_BO - Data Hora da Comunicação da Ocorrência
* FLAG_STATUS - Status da Ocorrência
* RUBRICA - Natureza jurídica da ocorrência
* DESCR_CONDUTA - Conduta na Ocorrência
* DESDOBRAMENTO - Desdobramento na Ocorrência
* DESCR_TIPOLOCAL - Tipo de Local
* DESCR_SUBTIPOLOCAL - Descrição do subtipo de local
* LOGRADOURO - Logradouro dos fatos
* NUMERO_LOGRADOURO - Numero do Logradouro dos fatos
* LATITUDE - Latitude da Ocorrência
* LONGITUDE - Longitude da Ocorrência
* DESCR_TIPO_PESSOA - Qualificação do envolvido na ocorrência
* SEXO_PESSOA - Sexo
* IDADE_PESSOA - Idade
* COR_CUTIS - Cor da Pele

#**1. Imports**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Diretamente do GitHub (dataset 2014):
url = 'https://raw.githubusercontent.com/debora28/pi-2020.2/main/2014-1-10mil.csv'
data = pd.read_csv(url)

#**2.1 Preparando os dados**

In [ ]:
print(data.isnull().sum())
print(data)

Unnamed: 0                   0
NUM_BO                       0
ANO_BO                       0
ID_DELEGACIA                 0
NOME_DEPARTAMENTO            0
NOME_SECCIONAL               0
DELEGACIA                    0
NOME_DEPARTAMENTO_CIRC       0
NOME_SECCIONAL_CIRC          0
NOME_DELEGACIA_CIRC          0
ANO                          0
MES                          0
DATA_OCORRENCIA_BO          87
HORA_OCORRENCIA_BO        1040
FLAG_STATUS                  0
RUBRICA                      0
DESDOBRAMENTO             9351
CONDUTA                   4835
LATITUDE                  1593
LONGITUDE                 1593
CIDADE                       0
LOGRADOURO                 429
NUMERO_LOGRADOURO         2467
FLAG_STATUS.1                0
DESCR_TIPO_PESSOA            0
CONT_PESSOA                  0
SEXO_PESSOA                  0
IDADE_PESSOA              1902
COR                       2372
DESCR_PROFISSAO           3002
DESCR_GRAU_INSTRUCAO      2832
dtype: int64
      Unnamed: 0  NUM_BO  

In [ ]:
data.shape

(10000, 31)

Retirando colunas que não serão usadas:

In [ ]:
data = data.drop(columns=['NOME_DEPARTAMENTO', 'NOME_SECCIONAL','NOME_DEPARTAMENTO_CIRC','NOME_SECCIONAL_CIRC','DELEGACIA','FLAG_STATUS','DESDOBRAMENTO','NUMERO_LOGRADOURO','FLAG_STATUS.1'])
data.to_csv('data_semDesdob.csv', index= False)
data.head()

,Unnamed: 0,NUM_BO,ANO_BO,ID_DELEGACIA,NOME_DELEGACIA_CIRC,ANO,MES,DATA_OCORRENCIA_BO,HORA_OCORRENCIA_BO,RUBRICA,CONDUTA,LATITUDE,LONGITUDE,CIDADE,LOGRADOURO,DESCR_TIPO_PESSOA,CONT_PESSOA,SEXO_PESSOA,IDADE_PESSOA,COR,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
0,243,2,2014,280401,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,NaN,NaN,NaN,BARUERI,R MARIA HELENA,Curador,7,F,51.0,Parda,AUXILIAR DE LIMPEZA,NaN
1,244,2,2014,280401,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,NaN,NaN,NaN,BARUERI,R MARIA HELENA,Indiciado,4,M,19.0,Parda,DESEMPREGADO,1 Grau completo
2,245,2,2014,280401,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,NaN,NaN,NaN,BARUERI,R MARIA HELENA,Testemunha,2,M,38.0,NaN,INVESTIGADOR(A),Superior completo
3,246,2,2014,280401,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,NaN,NaN,NaN,BARUERI,R MARIA HELENA,Testemunha,6,M,23.0,Parda,DESEMPREGADO,2 Grau completo
4,270,3,2014,30314,DEL.POL.ITAQUAQUECETUBA,2014,1,14/01/2014,17:00,Drogas,NaN,-2.348.960.369,-4.637.155.072,ITAQUAQUECETUBA,R UBATUBA,Testemunha,1,M,40.0,NaN,POLICIAL CIVIL,2 Grau completo


In [ ]:
data.isnull().sum().sum()

19685

In [ ]:
data.isnull().any()

Unnamed: 0              False
NUM_BO                  False
ANO_BO                  False
ID_DELEGACIA            False
NOME_DELEGACIA_CIRC     False
ANO                     False
MES                     False
DATA_OCORRENCIA_BO       True
HORA_OCORRENCIA_BO       True
RUBRICA                 False
CONDUTA                  True
LATITUDE                 True
LONGITUDE                True
CIDADE                  False
LOGRADOURO               True
DESCR_TIPO_PESSOA       False
CONT_PESSOA             False
SEXO_PESSOA             False
IDADE_PESSOA             True
COR                      True
DESCR_PROFISSAO          True
DESCR_GRAU_INSTRUCAO     True
dtype: bool

Preenchendo valores NaN ou NA com os valores mais frequentes:

In [ ]:
data['CONDUTA'].value_counts()

VEICULO                         1386
TRANSEUNTE                      1369
OUTROS                           681
RESIDENCIA                       422
INTERIOR DE VEICULO              354
ESTABELECIMENTO COMERCIAL        326
CARGA                            273
INTERIOR ESTABELECIMENTO         112
ESTABELECIMENTO BANCARIO          75
ESTABELECIMENTO-OUTROS            62
INTERIOR TRANSPORTE COLETIVO      46
ESTABELECIMENTO ENSINO            37
COLETIVO                          13
CONDOMINIO RESIDENCIAL             7
CONDOMINIO COMERCIAL               2
Name: CONDUTA, dtype: int64

In [ ]:
data['CONDUTA'].fillna('VEICULO', inplace=True)

In [ ]:
data['CONDUTA'].value_counts()

VEICULO                         6221
TRANSEUNTE                      1369
OUTROS                           681
RESIDENCIA                       422
INTERIOR DE VEICULO              354
ESTABELECIMENTO COMERCIAL        326
CARGA                            273
INTERIOR ESTABELECIMENTO         112
ESTABELECIMENTO BANCARIO          75
ESTABELECIMENTO-OUTROS            62
INTERIOR TRANSPORTE COLETIVO      46
ESTABELECIMENTO ENSINO            37
COLETIVO                          13
CONDOMINIO RESIDENCIAL             7
CONDOMINIO COMERCIAL               2
Name: CONDUTA, dtype: int64

In [ ]:
data['LATITUDE'].value_counts()

-2.371.286.115    48
-2.352.519.435    23
-2.364.027.263    23
-2.347.103.556    22
-2.355.850.851    22
                  ..
-2.367.241.228     1
-2.368.687.829     1
-2.350.842.368     1
-2.352.071.034     1
-2.365.265.065     1
Name: LATITUDE, Length: 3439, dtype: int64

In [ ]:
data['LATITUDE'].fillna('-23.71286115', inplace=True)

In [ ]:
data['LONGITUDE'].value_counts()

-4.678.186.771    48
-4.639.605.905    23
-4.681.568.811    23
-4.672.128.418    22
-4.661.519.738    22
                  ..
-4.670.165.643     1
-4.660.137.417     1
-4.648.568.583     1
-4.659.300.442     1
-4.679.164.516     1
Name: LONGITUDE, Length: 3439, dtype: int64

In [ ]:
data['LONGITUDE'].fillna('-46.78186771', inplace=True)

In [ ]:
data['DATA_OCORRENCIA_BO'].value_counts()

01/01/2014    2556
02/01/2014    1443
03/01/2014     911
06/01/2014     550
04/01/2014     415
              ... 
09/06/2014       1
29/05/2014       1
01/06/2014       1
23/05/2014       1
30/05/2014       1
Name: DATA_OCORRENCIA_BO, Length: 155, dtype: int64

In [ ]:
data['DATA_OCORRENCIA_BO'].fillna('01/01/2014', inplace=True)

In [ ]:
data['HORA_OCORRENCIA_BO'].value_counts()

12:00    239
15:00    218
16:00    207
14:00    204
01:00    201
        ... 
18:29      1
11:46      1
19:55      1
13:57      1
14:49      1
Name: HORA_OCORRENCIA_BO, Length: 484, dtype: int64

In [ ]:
data['HORA_OCORRENCIA_BO'].fillna('12:00', inplace=True)

In [ ]:
data['LOGRADOURO'].value_counts()

AV SAPOPEMBA                           65
EST DO M BOI MIRIM                     54
AV DO ESTADO                           39
R APA                                  35
AV CELSO GARCIA                        34
                                       ..
EST DO MARENGO                          1
RUA JOSE FERNANDES CALDAS               1
AVENIDA KEMEL ADDAS                     1
AVENIDA ENGENHEIRO GEORGE CORBISIER     1
R MARQUES DE REZENDE                    1
Name: LOGRADOURO, Length: 3186, dtype: int64

In [ ]:
data['LOGRADOURO'].fillna('AV SAPOPEMBA', inplace=True)

In [ ]:
data['COR'].value_counts()

Branca     4166
Parda      2517
Preta       575
Outros      334
Amarela      36
Name: COR, dtype: int64

In [ ]:
data['COR'].fillna('Branca', inplace=True)

In [ ]:
data['IDADE_PESSOA'].value_counts()

30.0     269
26.0     260
33.0     256
28.0     245
38.0     241
        ... 
103.0      1
87.0       1
91.0       1
89.0       1
88.0       1
Name: IDADE_PESSOA, Length: 92, dtype: int64

In [ ]:
data['IDADE_PESSOA'].fillna('30', inplace=True)

In [ ]:
data['DESCR_PROFISSAO'].value_counts()

POLICIAL MILITAR                827
POLICIAL CIVIL                  721
ESTUDANTE                       308
MOTORISTA                       300
DESEMPREGADO                    281
                               ... 
TECNICO EM QUIMICA                1
ENCADERNADOR(A)                   1
AUXILIAR TEC DE REFRIGERACAO      1
FISCAL TRANSPORTE COLETIVOS       1
TORNEIRO(A) MECANICO              1
Name: DESCR_PROFISSAO, Length: 417, dtype: int64

In [ ]:
data['DESCR_PROFISSAO'].fillna('POLICIAL MILITAR', inplace=True)

In [ ]:
data['DESCR_GRAU_INSTRUCAO'].value_counts()

2 Grau completo        2780
1 Grau completo        1882
Superior completo      1310
1 Grau incompleto       574
Superior incompleto     321
2 Grau incompleto       227
Analfabeto               74
Name: DESCR_GRAU_INSTRUCAO, dtype: int64

In [ ]:
data['DESCR_GRAU_INSTRUCAO'].fillna('2 Grau completo', inplace=True)

In [ ]:
print(data.isnull().any())
print()

Unnamed: 0              False
NUM_BO                  False
ANO_BO                  False
ID_DELEGACIA            False
NOME_DELEGACIA_CIRC     False
ANO                     False
MES                     False
DATA_OCORRENCIA_BO      False
HORA_OCORRENCIA_BO      False
RUBRICA                 False
CONDUTA                 False
LATITUDE                False
LONGITUDE               False
CIDADE                  False
LOGRADOURO              False
DESCR_TIPO_PESSOA       False
CONT_PESSOA             False
SEXO_PESSOA             False
IDADE_PESSOA            False
COR                     False
DESCR_PROFISSAO         False
DESCR_GRAU_INSTRUCAO    False
dtype: bool



In [ ]:
data.describe()

,Unnamed: 0,NUM_BO,ANO_BO,ID_DELEGACIA,ANO,MES,CONT_PESSOA
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000
mean,4999.50000,24.377100,2014.011400,76819.235900,2014.0,1.362000,2.625500
std,2886.89568,16.054192,0.112567,110985.447337,0.0,0.898909,1.934276
min,0.00000,1.000000,2014.000000,10004.000000,2014.0,1.000000,1.000000
25%,2499.75000,11.000000,2014.000000,10365.000000,2014.0,1.000000,1.000000
50%,4999.50000,21.000000,2014.000000,30211.000000,2014.0,1.000000,2.000000
75%,7499.25000,37.000000,2014.000000,140206.000000,2014.0,1.000000,3.000000
max,9999.00000,63.000000,2017.000000,990900.000000,2014.0,6.000000,17.000000


In [ ]:
data.dtypes

Unnamed: 0               int64
NUM_BO                   int64
ANO_BO                   int64
ID_DELEGACIA             int64
NOME_DELEGACIA_CIRC     object
ANO                      int64
MES                      int64
DATA_OCORRENCIA_BO      object
HORA_OCORRENCIA_BO      object
RUBRICA                 object
CONDUTA                 object
LATITUDE                object
LONGITUDE               object
CIDADE                  object
LOGRADOURO              object
DESCR_TIPO_PESSOA       object
CONT_PESSOA              int64
SEXO_PESSOA             object
IDADE_PESSOA            object
COR                     object
DESCR_PROFISSAO         object
DESCR_GRAU_INSTRUCAO    object
dtype: object

#**2.2 Convertendo dados categóricos em numéricos**
Codificação one-hot 



In [ ]:
novas_colunas = pd.get_dummies(data['RUBRICA']) 
data = pd.concat([data,novas_colunas], axis=1) 
data.head(2)

,Unnamed: 0,NUM_BO,ANO_BO,ID_DELEGACIA,NOME_DELEGACIA_CIRC,ANO,MES,DATA_OCORRENCIA_BO,HORA_OCORRENCIA_BO,RUBRICA,CONDUTA,LATITUDE,LONGITUDE,CIDADE,LOGRADOURO,DESCR_TIPO_PESSOA,CONT_PESSOA,SEXO_PESSOA,IDADE_PESSOA,COR,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO,Drogas,Estupro,Estupro de vulneravel,Furto,Furto qualificado,Homicidio,Homicidio culposo,Homicidio culposo por veiculo,Homicidio qualificado,Lesao corporal,Lesao corporal de natureza GRAVE,Lesao corporal seguida de morte,Lesao culposa por veiculo,Roubo
0,243,2,2014,280401,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,VEICULO,-23.71286115,-46.78186771,BARUERI,R MARIA HELENA,Curador,7,F,51,Parda,AUXILIAR DE LIMPEZA,2 Grau completo,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,244,2,2014,280401,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,VEICULO,-23.71286115,-46.78186771,BARUERI,R MARIA HELENA,Indiciado,4,M,19,Parda,DESEMPREGADO,1 Grau completo,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
data['RUBRICA'].value_counts()

Roubo                               3630
Lesao corporal                      1875
Drogas                              1694
Furto                               1012
Furto qualificado                    633
Lesao culposa por veiculo            595
Homicidio                            279
Estupro de vulneravel                103
Homicidio qualificado                 96
Estupro                               50
Homicidio culposo por veiculo         22
Lesao corporal seguida de morte        4
Lesao corporal de natureza GRAVE       4
Homicidio culposo                      3
Name: RUBRICA, dtype: int64

Criando uma coluna com o somatório dos casos de furto e roubo, excluindo outros tipos de crimes:

In [ ]:
data['CRIME'] = data['Roubo'] + data['Furto'] + data['Furto qualificado']
data.head(2)

,Unnamed: 0,NUM_BO,ANO_BO,ID_DELEGACIA,NOME_DELEGACIA_CIRC,ANO,MES,DATA_OCORRENCIA_BO,HORA_OCORRENCIA_BO,RUBRICA,CONDUTA,LATITUDE,LONGITUDE,CIDADE,LOGRADOURO,DESCR_TIPO_PESSOA,CONT_PESSOA,SEXO_PESSOA,IDADE_PESSOA,COR,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO,Drogas,Estupro,Estupro de vulneravel,Furto,Furto qualificado,Homicidio,Homicidio culposo,Homicidio culposo por veiculo,Homicidio qualificado,Lesao corporal,Lesao corporal de natureza GRAVE,Lesao corporal seguida de morte,Lesao culposa por veiculo,Roubo,CRIME
0,243,2,2014,280401,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,VEICULO,-23.71286115,-46.78186771,BARUERI,R MARIA HELENA,Curador,7,F,51,Parda,AUXILIAR DE LIMPEZA,2 Grau completo,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,244,2,2014,280401,01 D.P. BARUERI,2014,2,26/02/2014,17:00,Drogas,VEICULO,-23.71286115,-46.78186771,BARUERI,R MARIA HELENA,Indiciado,4,M,19,Parda,DESEMPREGADO,1 Grau completo,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Convertendo a coluna 'idade' em float:

In [ ]:
data['IDADE_PESSOA'] = data['IDADE_PESSOA'].astype('float')

In [ ]:
data.groupby(by='SEXO_PESSOA').size()

SEXO_PESSOA
F    2335
I     166
M    7499
dtype: int64

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(data['HORA_OCORRENCIA_BO'])
data['HORA_OCORRENCIA_BO'] = le.transform(data['HORA_OCORRENCIA_BO'])

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(data['ID_DELEGACIA'])
data['ID_DELEGACIA'] = le.transform(data['ID_DELEGACIA'])

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(data['CIDADE'])
data['CIDADE'] = le.transform(data['CIDADE'])

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(data['CONDUTA'])
data['CONDUTA'] = le.transform(data['CONDUTA'])

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(data['SEXO_PESSOA'])
data['SEXO_PESSOA'] = le.transform(data['SEXO_PESSOA'])

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(data['DESCR_PROFISSAO'])
data['DESCR_PROFISSAO'] = le.transform(data['DESCR_PROFISSAO'])

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(data['RUBRICA'])
data['RUBRICA'] = le.transform(data['RUBRICA'])

#**2.3 Separar a variável alvo**

In [ ]:
x = data[['HORA_OCORRENCIA_BO','ID_DELEGACIA','CIDADE','IDADE_PESSOA','CONDUTA','SEXO_PESSOA','DESCR_PROFISSAO']].copy()
y = data[['CRIME']].copy()

#**3. Dividir dados em teste e treinamento**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=3000)

#**4. Criar modelo de ML**

In [ ]:
tipos_crimes_classifier = DecisionTreeClassifier(max_leaf_nodes=7000, random_state=1)
tipos_crimes_classifier.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=7000,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [ ]:
print(tipos_crimes_classifier)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=7000,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')


#**5. Fazer predição**

In [ ]:
predictions = tipos_crimes_classifier.predict(x_test)

#**6. Testar acurácia**

In [ ]:
accuracy_score(y_true=y_test, y_pred=predictions)

0.8812121212121212